In [1]:
import pandas as pd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
train = pd.read_csv('C:\\Users\\Pablo\\MachineLearningTest\\data\\train.csv')

In [3]:
##Cabecera de la base train
train.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0


Cambiar valores de la columna "num"

In [4]:
##Lo que hacemos es reescribir los valores de la columna "num", el 0 por 0 y el 1, 2, 3 por 1
for i in range(len(train.num)):
    if train.num[i]!=0:
        train.loc[i,'num']=1

In [5]:
##Division de la base de datos "train" para probar el modelo
train = train.fillna(0)
X, y = train.iloc[:, :-1], train.iloc[:, -1]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

from sklearn.linear_model import LogisticRegression
log = LogisticRegression()
y_pred = log.fit(X_train,y_train).predict(X_test)

In [6]:
##MATRIZ DE CONFUSION DEL MODELO
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[40,  7],
       [10, 31]], dtype=int64)

In [7]:
##RESULTADO DE LA PREDICCION
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.80681818181818177

In [8]:
##Realizamos el uso de componentes principales para intentar mejorar el modelo
from sklearn.decomposition import PCA
pca = PCA(9)
Xtr =pca.fit(X).transform(X)
X_train, X_test, y_train, y_test = train_test_split(Xtr, y, test_size=0.33)
log = LogisticRegression()
y_pred = log.fit(X_train,y_train).predict(X_test)

In [9]:
##Matrix de confusion con nuestro modelo de 9 componentes principales
confusion_matrix(y_test, y_pred)

array([[48,  2],
       [ 5, 33]], dtype=int64)

In [10]:
##Resultado de la prediccion de nuestro modelo de 9 componentes principales
accuracy_score(y_test, y_pred)

0.92045454545454541

In [33]:
##Realizacion del modelo de prediccion con las variables C y K, para poder variar esos valores encontrar con cual obtenemos mejores resultados

from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

def traintest(c, k, pert_split=0.33):
    #result=[]
   
    pca = PCA(k)   
    Xtr =pca.fit(X).transform(X)
        
    X_train, X_test, y_train, y_test = train_test_split(Xtr, y, test_size=0.33, random_state=10)
    result_int=[]
        
        #result_int.append(X_train)
        #result_int.append(y_train)
        #result_int.append(X_test)
        #result_int.append(y_test)
        
    log = LogisticRegression(C=c)
    y_pred = log.fit(X_train,y_train).predict(X_test)
        
        #result_int.append(y_pred)
        
    mat_conf=confusion_matrix(y_test, y_pred)
        
    result_int.append(mat_conf)
        
    scor=accuracy_score(y_test, y_pred)
       
    result_int.append(scor)
        
    #result.append(result_int)
    
    return result_int

In [20]:
res= traintest(0.1, 9)
res[0]

array([[36,  9],
       [12, 31]], dtype=int64)

In [21]:
import itertools
def expandgrid(*itrs):
   product = list(itertools.product(*itrs))
   return {'Var{}'.format(i+1):[x[i] for x in product] for i in range(len(itrs))}

In [22]:
c= np.arange(0.1, 1, 0.05)
k= np.arange(6, 11, 1)
expandgrid(c, k)

parametres = pd.DataFrame.from_dict(expandgrid(c, k))
parametres.columns = ['C','k']

In [23]:
##Creamos una tabla con cada valor de C y K

TP = []
TN = []
FP = []
FN = []
AC = []
for i in range(parametres.shape[0]):
    result = traintest(parametres.loc[i,'C'],parametres.loc[i,'k'])
    confmat = result[0]
    TP.append(confmat[1, 1])
    TN.append(confmat[0, 0])
    FP.append(confmat[0, 1])
    FN.append(confmat[1, 0])
    AC.append(result[1])
parametres["TP"]=TP
parametres["TN"]=TN
parametres["FP"]=FP
parametres["FN"]=FN
parametres["AC"]=AC

In [24]:
parametres

,C,k,TP,TN,FP,FN,AC
0,0.10,6,28,35,10,15,0.715909
1,0.10,7,28,36,9,15,0.727273
2,0.10,8,31,37,8,12,0.772727
3,0.10,9,31,36,9,12,0.761364
4,0.10,10,31,37,8,12,0.772727
5,0.15,6,28,35,10,15,0.715909
6,0.15,7,29,36,9,14,0.738636
7,0.15,8,31,37,8,12,0.772727
8,0.15,9,31,37,8,12,0.772727
9,0.15,10,31,37,8,12,0.772727


In [30]:
parametres[parametres.AC>0.771]

,C,k,TP,TN,FP,FN,AC
2,0.10,8,31,37,8,12,0.772727
4,0.10,10,31,37,8,12,0.772727
7,0.15,8,31,37,8,12,0.772727
8,0.15,9,31,37,8,12,0.772727
9,0.15,10,31,37,8,12,0.772727
12,0.20,8,31,37,8,12,0.772727
13,0.20,9,31,37,8,12,0.772727
14,0.20,10,31,37,8,12,0.772727
17,0.25,8,31,37,8,12,0.772727
18,0.25,9,31,37,8,12,0.772727


In [80]:
test = pd.read_csv('C:\\Users\\Pablo\\MachineLearningTest\\data\\test.csv')

In [81]:
test.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,61.0,1.0,4.0,138.0,166.0,0.0,2.0,125.0,1.0,3.6,2.0,1.0,3.0
1,42.0,1.0,4.0,136.0,315.0,0.0,0.0,125.0,1.0,1.8,2.0,0.0,6.0
2,52.0,1.0,4.0,128.0,204.0,1.0,0.0,156.0,1.0,1.0,2.0,0.0,?
3,59.0,1.0,3.0,126.0,218.0,1.0,0.0,134.0,0.0,2.2,2.0,1.0,6.0
4,40.0,1.0,4.0,152.0,223.0,0.0,0.0,181.0,0.0,0.0,1.0,0.0,7.0


In [74]:
[i for i, el in enumerate(test.thal) if el=="?"]

[2]

In [82]:
a = []
for i, el in enumerate(test.ca):
    if el=="?":
        a.append(np.nan)
    else:
        a.append(float(el))
test.ca = a

In [83]:
b = []
for i, el in enumerate(test.thal):
    if el=="?":
        b.append(np.nan)
    else:
        b.append(float(el))
test.thal = b

In [84]:
test = test.fillna(test.mean())

In [85]:
test.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,61.0,1.0,4.0,138.0,166.0,0.0,2.0,125.0,1.0,3.6,2.0,1.0,3.000000
1,42.0,1.0,4.0,136.0,315.0,0.0,0.0,125.0,1.0,1.8,2.0,0.0,6.000000
2,52.0,1.0,4.0,128.0,204.0,1.0,0.0,156.0,1.0,1.0,2.0,0.0,5.131579
3,59.0,1.0,3.0,126.0,218.0,1.0,0.0,134.0,0.0,2.2,2.0,1.0,6.000000
4,40.0,1.0,4.0,152.0,223.0,0.0,0.0,181.0,0.0,0.0,1.0,0.0,7.000000


In [92]:
pca = PCA(8)   
Xtr=pca.fit(train).transform(train)
Xt=pca.fit(test).transform(test)
log = LogisticRegression()
y_pred = log.fit(Xtr,np.array(train.num)).predict(Xt)

In [93]:
y_pred

array([1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1,
       0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0], dtype=int64)

In [94]:
test['num'] = list(y_pred)

In [95]:
test.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
0,61.0,1.0,4.0,138.0,166.0,0.0,2.0,125.0,1.0,3.6,2.0,1.0,3.000000,1
1,42.0,1.0,4.0,136.0,315.0,0.0,0.0,125.0,1.0,1.8,2.0,0.0,6.000000,1
2,52.0,1.0,4.0,128.0,204.0,1.0,0.0,156.0,1.0,1.0,2.0,0.0,5.131579,1
3,59.0,1.0,3.0,126.0,218.0,1.0,0.0,134.0,0.0,2.2,2.0,1.0,6.000000,0
4,40.0,1.0,4.0,152.0,223.0,0.0,0.0,181.0,0.0,0.0,1.0,0.0,7.000000,1


In [96]:
y_prob = log.fit(Xtr,np.array(train.num)).predict_proba(Xt)

In [101]:
test["prob"] = y_prob[:,1]

In [102]:
test

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num,prob
0,61.0,1.0,4.0,138.0,166.0,0.0,2.0,125.0,1.0,3.6,2.0,1.000000,3.000000,1,0.908707
1,42.0,1.0,4.0,136.0,315.0,0.0,0.0,125.0,1.0,1.8,2.0,0.000000,6.000000,1,0.922905
2,52.0,1.0,4.0,128.0,204.0,1.0,0.0,156.0,1.0,1.0,2.0,0.000000,5.131579,1,0.568856
3,59.0,1.0,3.0,126.0,218.0,1.0,0.0,134.0,0.0,2.2,2.0,1.000000,6.000000,0,0.468730
4,40.0,1.0,4.0,152.0,223.0,0.0,0.0,181.0,0.0,0.0,1.0,0.000000,7.000000,1,0.676916
5,42.0,1.0,3.0,130.0,180.0,0.0,0.0,150.0,0.0,0.0,1.0,0.000000,3.000000,0,0.040517
6,61.0,1.0,4.0,140.0,207.0,0.0,2.0,138.0,1.0,1.9,1.0,1.000000,7.000000,1,0.941707
7,66.0,1.0,4.0,160.0,228.0,0.0,2.0,138.0,0.0,2.3,1.0,0.000000,6.000000,1,0.947020
8,46.0,1.0,4.0,140.0,311.0,0.0,0.0,120.0,1.0,1.8,2.0,2.000000,7.000000,1,0.918958
9,71.0,0.0,4.0,112.0,149.0,0.0,0.0,125.0,0.0,1.6,2.0,0.000000,3.000000,0,0.139371


In [105]:
test.to_csv('C:\\Users\\Pablo\\MachineLearningTest\\data\\resultsPablo.csv', index=False)